In [1]:
!gdown 18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
!unzip sentiment.zip
!rm sentiment.zip 
!ls

Downloading...
From (original): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_
From (redirected): https://drive.google.com/uc?id=18ZT8T7cUHvHNqoe_aZiDslR07lodEOK_&confirm=t&uuid=002299f5-b722-4652-b4b1-c03d06b907cf
To: /kaggle/working/sentiment.zip
100%|████████████████████████████████████████| 672M/672M [00:07<00:00, 85.1MB/s]
Archive:  sentiment.zip
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: NTC_SV/NTC_SV_test.csv  
  inflating: NTC_SV/NTC_SV_train.csv  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
multi_cased_L-12_H-768_A-12  NTC_SV


In [2]:
#wandb ()
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("bertbase-2") 

wandb.login(key=my_secret)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: storknguyen-286 (storknguyen-286-university-system-of-georgia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from safetensors.torch import save_file
import os
from tqdm import tqdm

2025-04-26 06:39:05.905599: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745649546.089689      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745649546.139428      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
SEQ_LEN = 256
BATCH_SIZE = 16
EPOCHS = 10
LR = 2e-5
SAVE_EVERY = 2

In [6]:
data = pd.read_csv("/kaggle/working/NTC_SV/NTC_SV_train.csv").dropna()
texts = data["review"].tolist()
labels = LabelEncoder().fit_transform(data["label"])

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [8]:
class FoodyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [9]:
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.15, random_state=42)
train_dataset = FoodyDataset(X_train, y_train, tokenizer, SEQ_LEN)
val_dataset = FoodyDataset(X_val, y_val, tokenizer, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [10]:
test = pd.read_csv("/kaggle/working/NTC_SV/NTC_SV_test.csv").dropna()
data_test = test["review"].tolist()
label_test = LabelEncoder().fit(test["label"]).transform(test["label"])
test_dataset = FoodyDataset(data_test, label_test, tokenizer, SEQ_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        return self.fc(x)

In [12]:
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 33.4 MB/s eta 0:00:00:00:0100:01


In [13]:
model = BERTClassifier(num_classes=len(set(labels))).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [14]:
os.makedirs("checkpoints", exist_ok=True)

In [15]:
import wandb

# Khởi tạo wandb
wandb.init(project="bert-base-2", name=f"run_epoch_{EPOCHS}", config={
    "epochs": EPOCHS,
    "model": "bert-base",
    "save_every": SAVE_EVERY,
    "optimizer": str(optimizer),
    "loss": str(criterion),
    "batch_size": BATCH_SIZE,
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [16]:
import wandb
from tqdm import tqdm
from safetensors.torch import save_file
from sklearn.metrics import accuracy_score

def train_model(model, train_loader, optimizer, criterion, EPOCHS, SAVE_EVERY, save_dir, device):
    # Khởi tạo wandb
    wandb.init(project="bert-rcnn", config={
        "epochs": EPOCHS,
        "batch_size": train_loader.batch_size,
        "optimizer": str(optimizer),
        "loss_fn": str(criterion),
    })

    global_step = 0

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []

        loop = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{EPOCHS}")

        for i, batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            batch_acc = (preds == labels).sum().item() / labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Log từng batch
            wandb.log({
                "batch_loss": loss.item(),
                "step": global_step
            })

            global_step += 1
            loop.set_postfix(batch_loss=loss.item(), batch_acc=batch_acc)

        # Tính và log trung bình theo epoch
        epoch_loss = total_loss / len(train_loader)
        epoch_acc = accuracy_score(all_labels, all_preds)

        wandb.log({
            "loss": epoch_loss,
            "accuracy": epoch_acc,
            "epoch": epoch + 1
        })

        print(f"✅ Epoch {epoch+1}: Loss = {epoch_loss:.4f}, Accuracy = {epoch_acc:.4f}")

        # Lưu checkpoint nếu cần
        if (epoch + 1) % SAVE_EVERY == 0:
            save_path = f"{save_dir}/bert_rcnn_epoch_{epoch+1}.safetensors"
            save_file(model.state_dict(), save_path)
            print(f"💾 Saved checkpoint at: {save_path}")
            wandb.save(save_path)

    wandb.finish()


In [17]:
train_model(
    model=model,
    train_loader=train_loader,
    optimizer=optimizer,
    criterion=criterion,
    EPOCHS=EPOCHS,
    SAVE_EVERY=SAVE_EVERY,
    save_dir="/kaggle/working",
    device=device
)

Epoch 1/10: 100%|██████████| 2166/2166 [16:43<00:00,  2.16it/s, batch_acc=0.833, batch_loss=0.207] 


✅ Epoch 1: Loss = 0.3309, Accuracy = 0.8583


Epoch 2/10: 100%|██████████| 2166/2166 [16:44<00:00,  2.16it/s, batch_acc=0.833, batch_loss=0.497] 


✅ Epoch 2: Loss = 0.2469, Accuracy = 0.9035


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


...

Epoch 9/10: 100%|██████████| 2166/2166 [16:42<00:00,  2.16it/s, batch_acc=1, batch_loss=0.0171]    


✅ Epoch 9: Loss = 0.0785, Accuracy = 0.9738


Epoch 10/10: 100%|██████████| 2166/2166 [16:43<00:00,  2.16it/s, batch_acc=1, batch_loss=0.00811]   


✅ Epoch 10: Loss = 0.0647, Accuracy = 0.9791
💾 Saved checkpoint at: /kaggle/working/bert_rcnn_epoch_10.safetensors


accuracy,▁▄▅▅▆▇▇▇██
batch_loss,█▄▄▂▃▃▃▄▇▄▃▂▁▂▅▂▆▄▃▂▇▁█▁▁▁▁▂▁▁▁▁▁▆▂▁▁▅▄▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▄▃▂▂▂▁▁
step,▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
accuracy,0.97907
batch_loss,0.00811
epoch,10
loss,0.06472
step,21659


In [18]:
model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        pred = torch.argmax(outputs, dim=1)
        preds.extend(pred.cpu().numpy())
        trues.extend(labels.cpu().numpy())

print("=== Test Result ===")
print(classification_report(trues, preds))


=== Test Result ===
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      5000
           1       0.90      0.88      0.89      5000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [19]:
save_file(model.state_dict(), "bert_base_foody.safetensors")